In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Embedding, LSTM, Input, Activation, Permute, Dropout, add, dot, concatenate
from keras.utils import to_categorical
from pickle import dump, load
from nltk.corpus import words
import nltk
import spacy
import keras

import numpy as np
import random
import re

random.seed(42)

In [2]:
nlp = spacy.load('en_core_web_sm')

In [2]:
%reset -f

## Load Dataset

In [3]:
with open('Datasets/tasks_1-20_v1-2/en-10k/qa6_yes-no-questions_test.txt', 'rb') as f:
    text_test = f.read().decode('utf-8')

with open('Datasets/tasks_1-20_v1-2/en-10k/qa6_yes-no-questions_train.txt', 'rb') as f:
    text_train = f.read().decode('utf-8')

In [6]:
text_test;

In [7]:
text_train;

## Data Preprocessing

In [8]:
def preprocessing_data(txt):
    data = []
    record = []
    temp = None
    start = True
    for sentence in txt.split('\n'):
        doc = nlp(sentence)
        token = [i.text for i in doc]
        if len(token) == 0:
            break

        if start:
            if temp:
                record.append(temp)
                record[0].extend(token[1:])
            else:
                record.append(token[1:])
            start = False
        elif '?' in token:
            record.append(token[1:-4])
            record.append(token[-3])
            temp = record[0].copy()
            data.append(record.copy())
            record = []
            start = True
        else:
            record[0].extend(token[1:])
    return data

In [11]:
train_data = preprocessing_data(text_train)
train_data;

In [9]:
test_data = preprocessing_data(text_test)
test_data;

## Create a Vocabulary to Store all Words

In [10]:
vocab = set()

In [12]:
merged_data = train_data + test_data

In [13]:
for story, question, _ in merged_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [14]:
vocab.add('yes')
vocab.add('no')
vocab;

In [15]:
VOCAB_LEN = len(vocab) + 1
VOCAB_LEN;

In [16]:
MAX_STORY_LEN = max(len(data[0]) for data in merged_data)
MAX_STORY_LEN;

In [17]:
MAX_QUESTION_LEN = max(len(data[1]) for data in merged_data)
MAX_QUESTION_LEN;

## Vectorize the Data

In [18]:
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [19]:
tokenizer.word_index;

In [20]:
# train_story_text = []
# train_question_text = []
# train_answers = []

# for story,question,answer in train_data:
#     train_story_text.append(story)
#     train_question_text.append(question)

# len(train_story_text)

In [21]:
# train_story_seq = tokenizer.texts_to_sequences(train_story_text)
# len(train_story_seq)

In [22]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=MAX_STORY_LEN, max_question_len=MAX_QUESTION_LEN):
    story_vectors = []
    question_vectors = []
    answer_indices = []  # Use indices instead of one-hot vectors

    for story, question, answer in data:
        # Vectorize the story and question
        story_vectors.append(
            [word_index.get(word.lower(), 0) for word in story[-max_story_len:]]
        )
        question_vectors.append(
            [word_index.get(word.lower(), 0) for word in question[-max_question_len:]]
        )
        
        # Store the answer as an index
        answer_indices.append(word_index.get(answer.lower(), 0))

    # Pad all sequences at once
    story_vectors = pad_sequences(story_vectors, maxlen=max_story_len)
    question_vectors = pad_sequences(question_vectors, maxlen=max_question_len)
    answer_indices = np.array(answer_indices)

    return story_vectors, question_vectors, answer_indices


In [23]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [24]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [25]:
inputs_train;

In [26]:
inputs_test;

In [27]:
answers_train = to_categorical(answers_train, num_classes=(len(tokenizer.word_index) + 1))
print(answers_train.shape)
answers_train;

(10000, 38)


In [28]:
answers_test = to_categorical(answers_test, num_classes=VOCAB_LEN)
print(answers_test.shape)
answers_test;

(1000, 38)


## Create End-to-End Memory Network (MemN2N) Model

### Input Placeholders for the Stories and Questions

In [29]:
input_sequence = Input((MAX_STORY_LEN, ))
question = Input((MAX_QUESTION_LEN, ))

### Encoders

In [30]:
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=VOCAB_LEN,output_dim=64))
input_encoder_m.add(Dropout(0.3))

In [31]:
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=VOCAB_LEN,output_dim=MAX_QUESTION_LEN))
input_encoder_c.add(Dropout(0.3))

In [32]:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=VOCAB_LEN, output_dim=64, input_length=MAX_QUESTION_LEN))
question_encoder.add(Dropout(0.3))

c:\Users\jayaw\.virtualenvs\Chatbot-Yes-No-Question-lJ-2E43I\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


### Encode the Sequences

In [33]:
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [34]:
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

In [35]:
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

In [36]:
answer = concatenate([response, question_encoded])
answer = LSTM(32)(answer)  # (samples, 32)
answer = Dropout(0.5)(answer)
answer = Dense(VOCAB_LEN)(answer)  # (samples, vocab_size)
answer = Activation('softmax')(answer)

In [37]:
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary();

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 126162)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 126162,    │      2,432 │ input_layer[0][0] │
│ (Sequential)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_2        │ (None, 6, 64)     │      2,432 │ input_layer_1[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 126162, 6) │          0 │ sequential[0][0], │
│                     │                   │            │ sequential_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 126162, 6) │          0 │ dot[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 126162, 6) │        228 │ input_layer[0][0] │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 126162, 6) │          0 │ activation[0][0], │
│                     │                   │            │ sequential_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ permute (Permute)   │ (None, 6, 126162) │          0 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 6, 126226) │          0 │ permute[0][0],    │
│ (Concatenate)       │                   │            │ sequential_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 32)        │ 16,161,152 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 32)        │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 38)        │      1,254 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 38)        │          0 │ dense[0][0]       │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 16,167,498 (61.67 MB)

 Trainable params: 16,167,498 (61.67 MB)

 Non-trainable params: 0 (0.00 B)

## Train the Model

In [ ]:
history = model.fit([inputs_train, queries_train], answers_train,batch_size=32,epochs=120,validation_data=([inputs_test, queries_test], answers_test))

Epoch 1/120
 18/313 ━━━━━━━━━━━━━━━━━━━━ 17:41 4s/step - accuracy: 0.2979 - loss: 2.6419

In [ ]:
filename = 'memn2n.h5'
model.save(filename)

In [ ]:
from google.colab import files
files.download(filename)